*for more information, checkout* [Github](https://github.com/Crazytoph/2Dcorrelation "my github page").

# CD Measurements - Data Analysis

## *setting up the Notebook*

In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
from scipy import integrate
import os

In [2]:
import analise as ana 
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [4]:
# change plot size
plt.rcParams["figure.figsize"] = (11,9)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [5]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data"
datalist = os.listdir(path)
print(datalist)

['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M']


In [6]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [7]:
# options to acces data
print(data_all['Control_0M'])
# multiple options
print(list(map(data_all.get, ['Control_0M', 'Gdm2SO4_4M'])))
print(tuple(data_all.values())[0:4])

[<cdata.CData object at 0x000001A63F8AD700>, <cdata.CData object at 0x000001A63F89D610>]
(<cdata.CData object at 0x000001A63F8AD700>, <cdata.CData object at 0x000001A63BF272B0>, <cdata.CData object at 0x000001A63F8ADF70>, <cdata.CData object at 0x000001A63F89D610>)


## Colormaps

In [8]:
plot.heatmap(*tuple(data_all.values()), subtitle=tuple(data_all.keys()), title="Cd-Values", x_min=[220]*len(data_all))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Function Plots 

In [9]:
data=[]
for key in data_all.keys():
    data.append([data_all[key]])

plot.mult_func([230, 247, 275, 300], *data, subtitle=tuple(data_all.keys()), title="Cd-Values")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Correlation Analyis

### Homogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [10]:
for key in data_all.keys():
    if key == "Control_0M":
        continue
    ref = data_all["Control_0M"].cd_df.loc[:, list(data_all[key].cd_df.columns)]    
    sync, assync = ana.correlation(data_all[key].cd_df, ref_spec=ref, scaling='pareto')
    plot.heatmap(sync, assync, title=str(key), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[220, 220], y_min=[220,220])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Sigmoid Fit and Melting Temperature

In [11]:
graph={}
melt_T={}
for key in data_all.keys():
    fit, params = ana.sigmoid_fit(ana.normalize(data_all[key].cd_df))
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = T
plot.mult_func([247, "fit", "deriv", "Maximum"], *graph.values(),  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.18677176 61.81965531] [0.01970753 0.53616699]
[ 0.24733216 63.42769918] [0.021931   0.35106747]
[ 0.3      63.952136] [0.02830562 0.31740654]
[ 0.28174123 64.87468031] [0.0257719  0.33422358]
[ 0.2364995  63.66483815] [0.01583725 0.27656029]
[ 0.29391682 61.61989348] [0.0191108  0.21232312]
[ 0.26911021 63.193142  ] [0.02238774 0.30450945]
[1.41708677e-02 5.00000000e+01] [1.82155660e-02 2.37101894e+01]
[2.29855156e-02 7.00000000e+01] [ 0.01983875 16.6558836 ]
[ 0.19424875 57.00576793] [0.02250772 0.5612431 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot the melting Temperatures for the Gdm2SO4 and GdmCl

In [12]:
x = [0., 0.5, 1., 2.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"]]
graph2 = pd.DataFrame(y2, index=x[0:3], columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Absorbance Values

In [13]:
absorbance = list(map(lambda x: data_all[x].absorb_df, list(data_all.keys())))
plot.heatmap(*absorbance, subtitle=tuple(data_all.keys()), title="Absorbance Values", x_min=[220]*len(data_all))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …